In [1]:
from torchkf import *
import torch
import numpy as np
import plotly.graph_objs as go
import plotly.express as px
import pykeos as pk

pyunicorn: Package netCDF4 could not be loaded. Some functionality in class Data might not be available!
pyunicorn: Package netCDF4 could not be loaded. Some functionality in class NetCDFDictionary might not be available!


In [2]:
hdm

NameError: name 'hdm' is not defined

In [15]:
def lorenz(x: torch.Tensor, v, P):
    x1 = (18. * x[1] - 18. * x[0])
    x2 = (46.92 * x[0] - 2 * x[2] * x[0] - x[1])
    x3 = (2 * x[0] * x[1] - 4 * x[2])
    return torch.stack([x1, x2, x3], dim=0)

models = [GaussianModel(
    f=lorenz, 
    g=lambda x, v, P: torch.ones((1, 3)) @ x,
    x=torch.tensor([12,13,16.]).unsqueeze(-1), 
    n=3, W=torch.tensor([np.exp(16)] * 3), V=torch.tensor([np.exp(16)]), 
)]

nT = 512
hdm = HierarchicalGaussianModel(*models)
gen = DEMInversion(hdm, states_embedding_order=6).generate(nT)
gen

{'v': tensor([[[[ 5.3087e+02]],
 
          [[ 2.5212e+04]],
 
          [[-1.1803e+05]],
 
          ...,
 
          [[-6.7299e+07]],
 
          [[ 1.6638e+08]],
 
          [[ 0.0000e+00]]],
 
 
         [[[ 1.2118e+17]],
 
          [[-3.5166e+17]],
 
          [[-7.9668e+21]],
 
          ...,
 
          [[ 4.9485e+26]],
 
          [[ 7.2243e+14]],
 
          [[ 0.0000e+00]]],
 
 
         [[[        nan]],
 
          [[        nan]],
 
          [[        nan]],
 
          ...,
 
          [[        nan]],
 
          [[        nan]],
 
          [[        nan]]],
 
 
         ...,
 
 
         [[[        nan]],
 
          [[        nan]],
 
          [[        nan]],
 
          ...,
 
          [[        nan]],
 
          [[        nan]],
 
          [[        nan]]],
 
 
         [[[        nan]],
 
          [[        nan]],
 
          [[        nan]],
 
          ...,
 
          [[        nan]],
 
          [[        nan]],
 
          [[        nan]]],
 
 
       

In [14]:
pk.SysWrapper(gen.x[:, 0, :, 0]).plot()

In [ ]:
y = torch.Tensor(np.sin(2 * np.pi * np.asarray([[5, 1]]) * np.linspace(0, 5, 500)[:, None] + 2* np.pi *np.random.randint(-1, 1, size=(1,2))))
measurements = y + 0.5* torch.randn(y.shape)

In [ ]:
y = pk.Lorenz(n_points=5000).states[:, :2]
measurements = torch.Tensor(y)+ 1* torch.randn(y.shape)
ll = []

In [ ]:
state_dim, obs_dim = 3, 1
# A = LinearTransform(torch.eye(3))
H = LinearTransform(torch.randn(2, 3))
state_space = GaussianStateSpaceModel(fwd_transform=f, obs_transform=g, state_dim=state_dim, obs_dim=obs_dim, 
                                      observation_noise_cov=1 * torch.eye(obs_dim), 
                                      process_noise_cov=np.exp(-8) *torch.eye(state_dim), initial_state_mean=torch.tensor([0.9, 0.8, 30.]))

In [ ]:
traj = state_space.blind_forecast(512)
plot_traj(traj['y'][None])
px.line(y=traj['y'].sample()[:,0])

In [ ]:
model_parameters = dict(fwd_transform=f, obs_transform=g, state_dim=state_dim, obs_dim=obs_dim, 
                                      observation_noise_cov=1 * torch.eye(obs_dim), 
                                      process_noise_cov=np.exp(-8) * torch.eye(state_dim), 
                                      initial_state_mean=torch.tensor([10.,10.,16.]))
# plot_traj(traj_filt['x_prior'])

In [ ]:
fig = go.Figure()

# y = traj['y'][None]

for i, initial_states in enumerate(([12,13,16], [8,0,30], [0, 10, 24], [5, 5,8], [17,2,35],  [5, 15,16])):
    y = state_space.sample(512)['y'][None]
    
    model_parameters['initial_state_mean'] = torch.tensor([float(v) for v in initial_states])
    fit_model = GaussianStateSpaceModel(**model_parameters)
    traj_filt = fit_model.filter(y, backward_pass=True)
    
    pk.SysWrapper(ts=traj_filt['x_prev'].mean[0, :, :2]).plot(fig=fig, show=False, 
                                                              line_color=px.colors.qualitative.Pastel[i+1], name=f'{initial_states}', legendgroup=f'{i+1}', opacity=0.7);
    fig.add_scatter(x=[initial_states[0]],y=[initial_states[1]], mode='markers', 
                    marker_color=px.colors.qualitative.Pastel[i+1], marker_size=10, legendgroup=f'{i+1}', showlegend=False)
    
# pk.SysWrapper(ts=traj_filt['x_prev'].mean[0, :, :2]).plot(fig=fig, show=False);

pk.SysWrapper(ts=traj['x'].mean[:, :2]).plot(fig=fig, show=False, line_color='black', line_dash='dash', name=f'Reference', opacity=0.8);
fig.add_scatter(x=[0.9],y=[0.8], mode='markers', marker_color='black', marker_size=10, showlegend=False)

fig.update_xaxes(range=(-20,24))
fig.update_yaxes(range=(-20,24))
fig.update_layout(height=600, width=600, template='simple_white', title={'text':'attractor reconstruction (EKF)',
         'x':0.5,
         'xanchor': 'center'})
fig.show()

In [ ]:
for i in range(10): 
    trajectory = state_space.filter(measurements[None, ...], backward_pass=True)
    A.fit(trajectory['x_backward'].mean[:, :-1], trajectory['x_backward'].mean[:, 1:])
    H.fit(trajectory['x_backward'].mean, measurements[None])
    state_space.fit_params(measurements[None], trajectory)    
    plot_traj(trajectory['x_backward']).show()
    plot_traj(trajectory['y_prior']).show()
    
    ll.append(state_space.complete_data_likelihood(measurements[None], trajectory))
    print(ll[-1])